## Работу выполнил Ковалев Евгений, студент 4 курса бакалавриата факультета математики НИУ ВШЭ

В данном задании нужно было попрактиковаться в работе с цепями Маркова для автоматической генерации текстов. Необходимо было написать код для генерации предложений на основе цепей Маркова порядка N (где N — натуральное число) и проверить его качество на тексте «Старик и море».

Класс Dictogram и функция make_markov_model были даны на семинаре, их я не менял.

https://tproger.ru/translations/markov-chains/

# Цепи Маркова для автоматической генерации тектов

## Структура данных Dictogram

Dictogram (dict — встроенный тип данных словарь в Python) будет отображать зависимость между звеньями и их частотой появления в тексте, то есть их распределение. Но при этом она будет обладать нужным нам свойством словаря — время выполнения программы не будет зависеть от объема входных данных, а это значит, мы создаем эффективный алгоритм.

In [2]:
import random

class Dictogram(dict):
    def __init__(self, iterable=None):
        # Инициализируем наше распределение как новый объект класса, 
        # добавляем имеющиеся элементы
        super(Dictogram, self).__init__()
        self.types = 0  # число уникальных ключей в распределении
        self.tokens = 0  # общее количество всех слов в распределении
        if iterable:
            self.update(iterable)

    def update(self, iterable):
        # Обновляем распределение элементами из имеющегося 
        # итерируемого набора данных
        for item in iterable:
            if item in self:
                self[item] += 1
                self.tokens += 1
            else:
                self[item] = 1
                self.types += 1
                self.tokens += 1

    def count(self, item):
        # Возвращаем значение счетчика элемента, или 0
        if item in self:
            return self[item]
        return 0

    def return_random_word(self):
        random_key = random.sample(self, 1)
        # Другой способ:
        # random.choice(histogram.keys())
        return random_key[0]

    def return_weighted_random_word(self):
        # Сгенерировать псевдослучайное число между 0 и (n-1),
        # где n - общее число слов
        random_int = random.randint(0, self.tokens-1)
        index = 0
#         list_of_keys = 
        # вывести 'случайный индекс:', random_int
        for key in self.keys():
            index += self[key]
            # вывести индекс
            if(index > random_int):
                # вывести list_of_keys[i]
                return key

В конструктор структуре Dictogram можно передать любой объект, по которому можно итерироваться. Элементами этого объекта будут слова для инициализации Dictogram, например, все слова из какой-нибудь книги. В данном случае мы ведем подсчет элементов, чтобы для обращения к какому-либо из них не нужно было пробегать каждый раз по всему набору данных.

Мы также сделали две функции для возврата случайного слова. Одна функция выбирает случайный ключ в словаре, а другая, принимая во внимание число появлений каждого слова в тексте, возвращает нужное нам слово.

## Cтруктура цепи Маркова N-го порядка

In [3]:
def make_markov_model(order, data):
    markov_model = dict()
    for i in range(0, len(data)-order):
        # Создаем окно
        window = tuple(data[i: i+order])
        # Добавляем в словарь
        if window in markov_model:
            # Присоединяем к уже существующему распределению
            markov_model[window].update([data[i+order]])
        else:
            markov_model[window] = Dictogram([data[i+order]])
    return markov_model

## Предобработка текста

In [4]:
import re

In [5]:
text = open('oldman.txt', 'r', encoding='utf-8').read()
text = re.sub('»|”|\.', ' END', text)
# переводим несколько последовательных вхождений END в одно
text = re.sub('( END)+', ' END', text)
# далее 'END' будет заменено на точку, что не имеет смысла, если перед ним стоит '?' либо '!'
text = re.sub('\? END', '?', text)
text = re.sub('! END', '!', text)
text = re.sub('«|“|,', '', text)
text = re.sub('\n', ' ', text)
frags = [f.lower() if f != 'END' else f for f in text.split(' ') if f]
print(' '.join(frags)[:2000])

he was an old man who fished alone in a skiff in the gulf stream and he had gone eighty-four days now without taking a fish END in the first forty days a boy had been with him END but after forty days without a fish the boy’s parents had told him that the old man was now definitely and finally salao which is the worst form of unlucky and the boy had gone at their orders in another boat which caught three good fish the first week END it made the boy sad to see the old man come in each day with his skiff empty and he always went down to help him carry either the coiled lines or the gaff and harpoon and the sail that was furled around the mast END the sail was patched with flour sacks and furled it looked like the flag of permanent defeat END the old man was thin and gaunt with deep wrinkles in the back of his neck END the brown blotches of the benevolent skin cancer the sun brings from its reflection on the tropic sea were on his cheeks END the blotches ran well down the sides of his fac

# Генерирование предложений

In [6]:
def generate_random_start(model):
    # порядок модели
    order = len(next(iter(model)))
    if order == 1:
        if ('END',) in model:
            return (model[('END',)].return_weighted_random_word(),)
    else:
        # генерируем корректные начальные кортежи, то есть те, что являлись началом предложений в корпусе
        end_keys = [t for t in list(model.keys()) if t[0] == 'END']
        if end_keys:
            random_int = random.randint(0, sum([sum(model[key].values()) for key in end_keys]) - 1)
            index = 0
            for key in end_keys:
                index += sum(model[key].values())
                if index > random_int:
                    return key
    return random.choice(list(model.keys()))


def generate_random_sentence(length, model):
    order = len(next(iter(model)))
    if order == 1:
        current_tuple = ('END',)
    else:
        current_tuple = generate_random_start(model)
    sentence = [current_tuple]
    i = 0
    while i < length:
        # если рассматриваемый кортеж является самым последним в тексте, то может возникнуть ошибка, ибо после него ничего нет
        try:
            current_dictogram = model[current_tuple]
            random_weighted_word = current_dictogram.return_weighted_random_word()
            current_tuple = current_tuple[1:] + (random_weighted_word,)
            sentence.append(current_tuple)
            i += 1
        except:
            # в случае ошибки генерируем случайный стартовый кортеж
            new_start = generate_random_start(model)
            j = 0
            # следим за тем, чтобы длина предложения не превысила заданный порог
            while j < order and i + j < length:
                sentence.append(current_tuple[j:] + new_start[:j])
                j += 1
            i += order
    sentence = [t[0] for t in sentence][1:]
    sentence[0] = sentence[0].capitalize()
    for i in range(1, len(sentence)):
        # все слова после знаков окончания предложения выводим с большой буквы
        try:
            if sentence[i - 1] == 'END' or sentence[i - 1][-1] in ['?', '!']:
                sentence[i] = sentence[i].capitalize()
        except:
            continue
    sentence = ' '.join(sentence).replace(' END', '.')
    # если в конце сгенерированного предложения нет знака окончания предложения, то нужно добавить точку
    if sentence[-1] in ['.', '?', '!']:
        return sentence
    else:
        return sentence + '.'

# Проверка

In [6]:
model = make_markov_model(3, frags)
st = generate_random_sentence(500, model)
print(st)
# вычитаем 1 и как правило получаем заданную длину, ибо в большинстве случаев в конец вывода добавляется лишняя точка
print(len(st.split()) + st.count('.') - 1)

I’ll eat some more for you. He ate the other part of the hook where the hand-forged hook projected from the head of the small tuna. The old man knew the breeze would last all night. The old man said. What did you catch? One the first day. One the second and two the third. Very good. If i cannot fish with you. I wish i could show him what a man can ask. Fish. He had sailed for two hours resting in the stern and lifted the carcass of the bonito by the tail and dropped it overboard. I don’t think i can eat an entire one. He said. Now i’ll let him eat it well. Eat it so that the small shrimps that were in it fell onto the planking of the skiff particles of phosphorus floated off and drifted slowly astern. He is beautiful and noble and knows no fear of anything. I killed him well. Besides he thought everything kills everything else in some way. Fishing kills me exactly as it keeps me alive. The boy said. Drink this. He said to himself. During the day he had taken as he died. The skiff was s

In [7]:
model = make_markov_model(8, frags)
st = generate_random_sentence(500, model)
print(st)
print(len(st.split()) + st.count('.') - 1)

The two sharks closed together and as he saw the one nearest him open his jaws and sink them into the silver side of the fish he raised the club high and brought it down heavy and slamming onto the top of the shark’s broad head. He felt the rubbery solidity as the club came down. But he felt the rigidity of bone too and he struck the shark once more hard across the point of the nose as he slid down from the fish. The other shark had been in and out and now came in again with his jaws wide. The old man could see pieces of the meat of the fish spilling white from the corner of his jaws as he bumped the fish and closed his jaws. He swung at him and hit only the head and the shark looked at him and wrenched the meat loose. The old man swung the club down on him again as he slipped away to swallow and hit only the heavy solid rubberiness. Come on galano. The old man said. Come in again. The shark came in a rush and the old man hit him as he shut his jaws. He hit him solidly and from as high

In [8]:
model = make_markov_model(1, frags)
st = generate_random_sentence(500, model)
print(st)
print(len(st.split()) + st.count('.') - 1)

But if a week. At his back of line stretching ahead if it up. He was clear as he found a man i could not care for all sharks hit the fish must have a long green turtles eating the water and all i clean it. In the wall and shoulders and started to eat them in the yankees of the water in time he had seen or i am sorry about the salt water. Now. They leaped. I had cut into the old man said. But i am sorry for now eating had to sleep. It once and to say. Is rougher where they had come again. That they came to keep his neck was santiago. I will see if there was barefooted. He tried to have left from the coast was still an hour it down on his left hand resting on it. Her aboard the boy and mine and it had the cast net and rested against his wife. He rammed the side to a virtue not trying with the wide silver belly up measuring the breeze brought it was inside. Yes. But suffering as they came to have never carried a piece and respected it as badly. The bird dipped again but it out. I will hur

In [11]:
model = make_markov_model(15, frags)
st = generate_random_sentence(500, model)
print(st)
print(len(st.split()) + st.count('.') - 1)

The fisherman who was measuring him called. I believe it. The boy said. He went into the terrace and asked for a can of coffee. Hot and with plenty of milk and sugar in it. Anything more? No. Afterwards i will see what he can eat. What a fish it was. The proprietor said. There has never been such a fish. Those were two fine fish you took yesterday too. Damn my fish. The boy said and he started to cry again. Do you want a drink of any kind? The proprietor asked. No. The boy said. Tell them not to bother santiago. I’ll be back. Tell him how sorry i am. Thanks. The boy said. The boy carried the hot can of coffee up to the old man’s shack and sat by him until he woke. Once it looked as though he were waking. But he had gone back into heavy sleep and the boy had gone across the road to borrow some wood to heat the coffee. Finally the old man woke. Don’t sit up. The boy said. Drink this. He poured some of the coffee in a glass. The old man took it and drank it. They beat me manolin. He said.

Как видно, все работает отлично, из четырех построенных моделей для порядка 8 получилось даже что-то достаточно осмысленное.